# Część!

## Jak poprawnie odpalić ten notebook na datasecie testowym?

1. Przygotuj folder z obrazkami testowymi za pomocą schematu:
```
| - submission.ipynb
|
└───test_data
    ├───barszcz
    ├───bigos
    ├───grzybowa
    ├───Kutia
    ├───makowiec
    ├───piernik
    ├───pierogi
    └───sernik
└───models
    ├───mobilenet_v3_large_best_f1.pth
    ├───efficientnet_b0_best_f1.pth
    ├───efficientnet_b1_best_f1.pth
    ├───shufflenet_v2_x2_0_best_f1.pth
```

Proszę pilnuj tego aby nazwy folderów z klasami byli takie same jak w stukturze wyżej!

2. Umieść obrazki każdej klasy w odpowidającym folderze.

3. Aby urochomić poniższy kod będziesz potrzebował bibliotek z requirements.txt

Możesz ich zainstalować komendą: `pip install -r requirements.txt`

4. Potrzebujesz też CUDA 12.1

5. Puść poniższy kod!

In [32]:
import random
import torch.nn as nn
import torch
from torchvision import datasets, transforms, models
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"


print(device)

cuda


In [33]:
def set_seed(seed_value):
    random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [34]:
def load_test_data(root_dir="test_data"):
    size_tuple = (224, 224)
    # Define a mapping from folder names to class numbers
    class_mapping = {
        "barszcz": 1,
        "bigos": 2,
        "Kutia": 3,
        "makowiec": 4,
        "piernik": 5,
        "pierogi": 6,
        "sernik": 7,
        "grzybowa": 8,
    }

    batch_size = 32

    # Load the original dataset
    test_dataset = datasets.ImageFolder(
        root=root_dir,
        transform=transforms.Compose(
            [
                transforms.Resize(size_tuple),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),  # Normalize
            ]
        ),
    )

    test_dataset.class_to_idx = {
        k: class_mapping[k] for k in test_dataset.class_to_idx.keys()
    }

    # Create data loaders
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    return test_loader

In [35]:
from collections import defaultdict
from sklearn.metrics import classification_report
from torch.utils.data import Subset


class EnsembleTrainer:
    def __init__(self, num_classes, device):
        self.device = device
        self.model_weights = dict()

        self.models = {
            "efficientnet_b0": self._prepare_efficientnet_b0(
                num_classes, "efficientnet_b0"
            ),
            "efficientnet_b1": self._prepare_efficientnet_b1(
                num_classes, "efficientnet_b1"
            ),
            "mobilenet_v3_large": self._prepare_mobilenet_v3_large(
                num_classes, "mobilenet_v3_large"
            ),
            "shufflenet_v2_x2_0": self._prepare_shufflenet_v2_x2_0(
                num_classes, "shufflenet_v2_x2_0"
            ),
        }

    def _prepare_efficientnet_b0(self, num_classes, model_name):
        model = models.efficientnet_b0(weights="DEFAULT")
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

        # Load the saved checkpoint
        checkpoint = torch.load(f"models/{model_name}_best_f1.pth")

        # Load the model weights
        model.load_state_dict(checkpoint["model_state_dict"])

        # Optional: Set the model to evaluation mode
        model.eval()
        self.model_weights[model_name] = checkpoint["best_val_f1"]
        return model.to(self.device)

    def _prepare_efficientnet_b1(self, num_classes, model_name):
        model = models.efficientnet_b1(weights="IMAGENET1K_V2")
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

        # Load the saved checkpoint
        checkpoint = torch.load(f"models/{model_name}_best_f1.pth")

        # Load the model weights
        model.load_state_dict(checkpoint["model_state_dict"])

        # Optional: Set the model to evaluation mode
        model.eval()
        self.model_weights[model_name] = checkpoint["best_val_f1"]
        return model.to(self.device)

    def _prepare_mobilenet_v3_large(self, num_classes, model_name):
        model = models.mobilenet_v3_large(weights="IMAGENET1K_V2")
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

        # Load the saved checkpoint
        checkpoint = torch.load(f"models/{model_name}_best_f1.pth")

        # Load the model weights
        model.load_state_dict(checkpoint["model_state_dict"])

        # Optional: Set the model to evaluation mode
        model.eval()
        self.model_weights[model_name] = checkpoint["best_val_f1"]

        return model.to(self.device)

    def _prepare_shufflenet_v2_x2_0(self, num_classes, model_name):
        model = models.shufflenet_v2_x2_0(weights="DEFAULT")
        model.fc = nn.Linear(model.fc.in_features, num_classes)

        # Load the saved checkpoint
        checkpoint = torch.load(f"models/{model_name}_best_f1.pth")

        # Load the model weights
        model.load_state_dict(checkpoint["model_state_dict"])

        # Optional: Set the model to evaluation mode
        model.eval()
        self.model_weights[model_name] = checkpoint["best_val_f1"]

        return model.to(self.device)

    def sequential_batch_evaluation(self, dataloader, batch_size=20):
        # Total number of classes
        num_classes = len(dataloader.dataset.classes)

        # Create a list to track F1 scores for each sequential batch
        sequential_f1_scores = []

        # Get the total number of images
        total_images = len(dataloader.dataset)

        # Calculate how many full batches of 10 images per class we can process
        images_per_class = total_images // num_classes
        batches_per_class = images_per_class // batch_size

        # Iterate through sequential batches
        for batch_group in range(batches_per_class):
            # Reset ensemble probabilities and labels for this batch group
            all_ensemble_probs = defaultdict(list)
            all_labels = []

            # Process 10 images from each class
            for class_idx in range(num_classes):
                start_idx = (batch_group * batch_size) + (class_idx * images_per_class)
                end_idx = start_idx + batch_size

                # Create a subset for this specific slice of images
                subset_indices = list(range(start_idx, end_idx))
                subset = Subset(dataloader.dataset, subset_indices)

                # Create a new dataloader for this subset
                subset_loader = torch.utils.data.DataLoader(
                    subset, batch_size=batch_size, shuffle=False
                )

                # Process this subset
                for images, labels in subset_loader:
                    images = images.to(self.device)

                    # Collect probabilities from each model
                    for model_name, model in self.models.items():
                        probs = torch.softmax(model(images), dim=1)
                        all_ensemble_probs[model_name].append(probs.detach().cpu())

                    all_labels.extend(labels.cpu().numpy())

            # Normalize model weights
            n = 5
            self.model_weights_now = {k: v**n for k, v in self.model_weights.items()}
            total_weight = sum(self.model_weights_now.values())
            model_weights_normalized = {
                model: weight / total_weight
                for model, weight in self.model_weights_now.items()
            }

            # Calculate ensemble predictions
            all_ensemble_preds = []
            for i in range(len(list(all_ensemble_probs.values())[0])):
                weighted_preds = []
                for model_name in list(self.models.keys()):
                    weight = model_weights_normalized.get(model_name, 0.0)
                    if weight == 0.0:
                        print(f"Warning: No weight for model {model_name}")
                        break
                    weighted_preds.append(all_ensemble_probs[model_name][i] * weight)

                ensemble_probs = torch.sum(torch.stack(weighted_preds), dim=0)
                _, ensemble_pred = torch.max(ensemble_probs, 1)
                all_ensemble_preds.extend(ensemble_pred.cpu().numpy())

            # Calculate F1 score for this batch group
            batch_f1 = f1_score(all_labels, all_ensemble_preds, average="weighted")
            sequential_f1_scores.append(batch_f1)

            # Print detailed report for this batch group
            print(f"\nBatch Group {batch_group + 1} Results:")
            print(f"Weighted Ensemble F1 Score: {batch_f1}")
            print("\nDetailed Classification Report:")
            print(
                classification_report(
                    all_labels, all_ensemble_preds, digits=4, zero_division=0
                )
            )

        # Print overall summary of F1 scores
        print("\nSequential Batch F1 Scores:")
        print(sequential_f1_scores)
        print(f"Average F1 Score: {np.mean(sequential_f1_scores)}")
        print(f"F1 Score Standard Deviation: {np.std(sequential_f1_scores)}")

        return sequential_f1_scores

In [36]:
test_loader = load_test_data("test_data")

ensembler = EnsembleTrainer(num_classes=8, device=device)
our_results = ensembler.sequential_batch_evaluation(test_loader)

C:\Users\PC\AppData\Local\Temp\ipykernel_5096\1608705122.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'models/{model_name}_best_f1.pth')
C:\


Batch Group 1 Results:
Weighted Ensemble F1 Score: 0.8156297031855285

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.7143    0.7500    0.7317        20
           1     0.7692    1.0000    0.8696        20
           2     1.0000    0.9000    0.9474        20
           3     0.7727    0.8500    0.8095        20
           4     0.8421    0.8000    0.8205        20
           5     0.8261    0.9500    0.8837        20
           6     0.8235    0.7000    0.7568        20
           7     0.8571    0.6000    0.7059        20

    accuracy                         0.8187       160
   macro avg     0.8256    0.8187    0.8156       160
weighted avg     0.8256    0.8187    0.8156       160


Batch Group 2 Results:
Weighted Ensemble F1 Score: 0.8034503280928993

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.9048    0.9500    0.9268        20
           1     0.6522    0.750